In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [26]:
# Load the data
data = pd.read_csv('creditcard.csv')

In [27]:
scaler = StandardScaler()
data['scaled_amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1,1))
data.drop(['Time', 'Amount'], axis=1, inplace=True)

In [28]:
# Split the data into train and test sets
X = data.drop(['Class'], axis=1)
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [29]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(29,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
# Reshape the data for CNN
X_train_cnn = np.reshape(X_train.values, (X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = np.reshape(X_test.values, (X_test.shape[0], X_test.shape[1], 1))

In [31]:
# Train the CNN model
model.fit(X_train_cnn, y_train, epochs=5, batch_size=32)


Epoch 1/5
6231/6231 [==============================] - 28s 4ms/step - loss: 0.0063 - accuracy: 0.9991
Epoch 2/5
6231/6231 [==============================] - 27s 4ms/step - loss: 0.0037 - accuracy: 0.9994
Epoch 3/5
6231/6231 [==============================] - 26s 4ms/step - loss: 0.0036 - accuracy: 0.9994
Epoch 4/5
6231/6231 [==============================] - 27s 4ms/step - loss: 0.0035 - accuracy: 0.9993
Epoch 5/5
6231/6231 [==============================] - 27s 4ms/step - loss: 0.0033 - accuracy: 0.9994


In [32]:
# Extract features using CNN
cnn_features = model.predict(X_train_cnn)

6231/6231 [==============================] - 13s 2ms/step


In [33]:
# Train the SVM model using the CNN features
svm_model = SVC(kernel='linear')
svm_model.fit(cnn_features, y_train)


SVC(kernel='linear')

In [34]:
# Extract features for test set
cnn_features_test = model.predict(X_test_cnn)

2671/2671 [==============================] - 6s 2ms/step


In [35]:
# Predict using the SVM model
svm_pred = svm_model.predict(cnn_features_test)

In [36]:
# Print classification report and confusion matrix
print(classification_report(y_test, svm_pred))
print(confusion_matrix(y_test, svm_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.82      0.82      0.82       136

    accuracy                           1.00     85443
   macro avg       0.91      0.91      0.91     85443
weighted avg       1.00      1.00      1.00     85443

[[85283    24]
 [   25   111]]


In [37]:
acc_sc = accuracy_score(y_test, y_pred=svm_pred)
print(acc_sc*100)

99.94265182636377
